In [2]:
import pyodbc

## SQL Server Connection

In [9]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try: 
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [ ]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'master'#yeni bir db oluşturacağız
user = 'sa' 
password = '------'
conn = create_server_connection(driver, server, database, user, password)

In [3]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

### windows authentication

In [4]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [55]:
conn = pyodbc.connect(conn_string);

In [56]:
crs = conn.cursor()

In [57]:
conn.autocommit = True

In [9]:
create_database_query = 'Create database test_1'

In [17]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try: 
        crs.execute(create_database_query)
        print("Database is Created Succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [11]:
create_database(conn, create_database_query)

Database is Created Succesfully


In [18]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [19]:
query = 'USE Test_1'
execute_query(conn,query)

Query Succeessful!


## Create Table 

In [ ]:
query = 'CREATE TABLE TestA (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

execute_query(conn, query)

## Insert

In [21]:
crs= conn.cursor()
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")
#conn.commit()

In [22]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bruce', 'Lee')

In [23]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

In [24]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

## Delete 

In [21]:
crs.execute("DELETE FROM TestA WHERE FirstName = 'Jerry'")

In [22]:
crs.execute("DELETE FROM TestA WHERE FirstName = ?" ,'Stefan')

## Rowcount 

In [23]:
crs.rowcount

1

In [24]:
print(crs.rowcount,"row(s) deleted")

1 row(s) deleted


## read data

### fetchone()

In [25]:
crs.execute('SELECT FirstName, LastName FROM TestA')
row = crs.fetchone() 
row

('Bob', 'Marley')

In [26]:
row = crs.fetchone() 
row

('Bruce', 'Lee')

In [27]:
row.FirstName

'Bruce'

In [28]:
row.LastName

'Lee'

## fetchall()

In [29]:
crs.execute('SELECT ID, FirstName, LastName FROM TestA')
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Hanks'),
 (6, 'Bart', 'Simpson'),
 (7, 'Tom', 'Cat'),
 (9, 'Owen', 'William'),
 (11, 'Bob', 'Marley'),
 (12, 'Bruce', 'Lee'),
 (13, 'Tom', 'Hanks'),
 (14, 'Bart', 'Simpson'),
 (15, 'Tom', 'Cat'),
 (16, 'Jerry', 'Mouse'),
 (17, 'Owen', 'William'),
 (18, 'Stefan', 'Müller')]

In [30]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Tom Hanks
6 Bart Simpson
7 Tom Cat
9 Owen William
11 Bob Marley
12 Bruce Lee
13 Tom Hanks
14 Bart Simpson
15 Tom Cat
16 Jerry Mouse
17 Owen William
18 Stefan Müller


In [31]:
crs.execute("""SELECT ID, FirstName, LastName 
                FROM TestA""")
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Hanks'),
 (6, 'Bart', 'Simpson'),
 (7, 'Tom', 'Cat'),
 (9, 'Owen', 'William'),
 (11, 'Bob', 'Marley'),
 (12, 'Bruce', 'Lee'),
 (13, 'Tom', 'Hanks'),
 (14, 'Bart', 'Simpson'),
 (15, 'Tom', 'Cat'),
 (16, 'Jerry', 'Mouse'),
 (17, 'Owen', 'William'),
 (18, 'Stefan', 'Müller')]

In [32]:
type(rows)

list

In [33]:
crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestA
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows

[(9, 'Owen', 'William'), (17, 'Owen', 'William')]

## Fetchmany()

In [34]:
crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestA""")
rows = crs.fetchmany(2)
rows

[(1, 'Bob', 'Marley'), (2, 'Bruce', 'Lee')]

## Fetchval()

In [35]:
crs.execute("SELECT MAX(ID) FROM TestA").fetchval()

9

In [36]:
crs.execute("SELECT FirstName FROM TestA").fetchval()

'Bob'

## Pandas Methods

In [34]:
import pandas as pd

In [35]:
def read_query (conn, query): 
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [36]:
query = """SELECT ID, FirstName, LastName FROM TestA"""

In [37]:
result = read_query (conn, query)

In [38]:
result

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Hanks'),
 (6, 'Bart', 'Simpson'),
 (7, 'Tom', 'Cat'),
 (9, 'Owen', 'William'),
 (11, 'Bob', 'Marley'),
 (12, 'Bruce', 'Lee'),
 (13, 'Tom', 'Hanks'),
 (14, 'Bart', 'Simpson'),
 (15, 'Tom', 'Cat'),
 (16, 'Jerry', 'Mouse'),
 (17, 'Owen', 'William'),
 (18, 'Stefan', 'Müller')]

In [39]:
df= pd.DataFrame(result)

In [40]:
df

,0
0,"[1, Bob, Marley]"
1,"[2, Bruce, Lee]"
2,"[3, Tom, Hanks]"
3,"[4, Bart, Simpson]"
4,"[5, Tom, Hanks]"
5,"[6, Bart, Simpson]"
6,"[7, Tom, Cat]"
7,"[9, Owen, William]"
8,"[11, Bob, Marley]"
9,"[12, Bruce, Lee]"


In [41]:
from_db = []

for result in result:
    result = list(result)
    from_db.append(result)

In [42]:
from_db

[[1, 'Bob', 'Marley'],
 [2, 'Bruce', 'Lee'],
 [3, 'Tom', 'Hanks'],
 [4, 'Bart', 'Simpson'],
 [5, 'Tom', 'Hanks'],
 [6, 'Bart', 'Simpson'],
 [7, 'Tom', 'Cat'],
 [9, 'Owen', 'William'],
 [11, 'Bob', 'Marley'],
 [12, 'Bruce', 'Lee'],
 [13, 'Tom', 'Hanks'],
 [14, 'Bart', 'Simpson'],
 [15, 'Tom', 'Cat'],
 [16, 'Jerry', 'Mouse'],
 [17, 'Owen', 'William'],
 [18, 'Stefan', 'Müller']]

In [43]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(from_db, columns = columns)
df

,ID,FirstName,LastName
0,1,Bob,Marley
1,2,Bruce,Lee
2,3,Tom,Hanks
3,4,Bart,Simpson
4,5,Tom,Hanks
5,6,Bart,Simpson
6,7,Tom,Cat
7,9,Owen,William
8,11,Bob,Marley
9,12,Bruce,Lee


In [44]:
query = 'USE BikeStores'
execute_query(conn,query)

Query Succeessful!


In [45]:
crs.execute("SELECT * FROM production.brands").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Heller'),
 (4, 'Pure Cycles'),
 (5, 'Ritchey'),
 (6, 'Strider'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [46]:
pd.read_sql("SELECT * FROM production.brands", con = conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [47]:
df_prod = pd.read_sql("SELECT * FROM production.products", con = conn)
df_prod.head(10)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,8,6,2016,469.99
6,7,Trek Slash 8 27.5 - 2016,9,6,2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,9,6,2016,1799.99
8,9,Trek Conduit+ - 2016,9,5,2016,2999.99
9,10,Surly Straggler - 2016,8,4,2016,1549.00


## DF to Databese

In [63]:
import sqlalchemy
import urllib

In [64]:
crs.execute('USE Test_1')

In [65]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'Test_1'#yeni bir db oluşturacağız
user = 'sa' 
password = '---'

In [66]:
params = 'DRIVER='+driver + ';SERVER=' + server +';PORT=1433;DATABASE='+ database + ';UID=' + user + ';PWD=' + password 

In [67]:
db_params = urllib.parse.quote_plus (params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [ ]:
df_prod = pd.read_sql("SELECT * FROM production.products where product_id < 10", con = conn)
df_prod.head(10)

In [ ]:
df_prod.to_sql("products_bikes", engine, index = False, if_exists = "append", schema = "dbo")